In [ ]:
import sys
import logging
import gym
import importlib

from cyberbattle._env.defender import ScanAndReimageCompromisedMachines
from cyberbattle._env.cyberbattle_env import AttackerGoal, DefenderConstraint
import cyberbattle.agents.baseline.plotting as p
sys.path.insert(0, '/Users/freddiebarrell/Desktop/Repository/Sarsa(Lambda)')
import agent_sarsa_lambda as a
import agent_wrapper as w
import learner as learner
from agent_wrapper import Verbosity

importlib.reload(learner)
importlib.reload(p)

logging.basicConfig(stream=sys.stdout, level=logging.ERROR, format="%(levelname)s: %(message)s")

In [ ]:
gym_env = gym.make("CyberBattleToyCtf-v0",
                   attacker_goal=AttackerGoal(
                       #reward=430,
                       own_atleast_percent=1),
                  defender_constraint=DefenderConstraint(
                                         maintain_sla=0.80
                                     ),
                                     defender_agent=ScanAndReimageCompromisedMachines(
                                         probability=0.6,
                                         scan_capacity=2,
                                         scan_frequency=5))

ep = w.EnvironmentBounds.of_identifiers(
    maximum_node_count=12,
    maximum_total_credentials=10,
    identifiers=gym_env.identifiers
)

In [ ]:
iteration_count = 1000
training_episode_count = 15

In [ ]:
sarsa_lambda_with_defender = learner.epsilon_greedy_search(
    gym_env,
    ep,
    learner=a.SarsaLambdaPolicy(
        ep,
        gamma=0.015, λ=0.5, learning_rate=0.01, hash_size=98689),
    episode_count=training_episode_count,
    iteration_count=iteration_count,
    epsilon=0.9,
    render=False,
    epsilon_exponential_decay=1000,
    epsilon_minimum=0.01,
    verbosity=Verbosity.Quiet,
    title="Sarsa_Lambda with defender"
)

In [ ]:
sarsa_lambda_exploit_with_defender = learner.epsilon_greedy_search(
gym_env,
ep,
learner=sarsa_lambda_with_defender['learner'],
episode_count=10,
iteration_count=1000,
epsilon=0,
render=False,
epsilon_minimum=0,
verbosity=Verbosity.Quiet,
title="Exploiting Sarsa_lambda with defender"
)